# Modélisation des forces perturbatrices en orbite terrestre basse
***Application à la Station Spatiale Internationale***

![](resources/iss_banner.jpg)

Assemblée à partir de 1998, la **Station spatiale internationale (ISS)** est actuellement le plus grand des objets artificiels placés en orbite par l'Homme, avec une masse d'environ 420 tonnes pour une longueur totale de près de 110 mètres [[1]](#ref-1). Ce statut fait d'elle un objet de premier intérêt dès lors que l'on s'intéresse au comportement des corps situés en orbite basse terrestre. En effet, à une altitude d'environ 400 km au dessus de la surface de la Terre, les perturbations du mouvement intertiel sont nombreuses, et l'objectif de ce calepin est de les étudier.

Dans ce projet, nous cherchons à identifier et modéliser les forces prédominantes agissant sur les objets placés en orbite basse terrestre. L'objectif d'un tel modèle est de prioriser et de ne retenir que les forces prédominantes, afin d'expliquer au mieux la trajectoire de la station spatiale internationale. Dans une première partie, nous présentons notre modèle et les forces retenues. Nous utilisons ensuite des données de positions mises à disposition par la NASA pour confronter les résultats de notre modèle. Enfin, nous discutons ces résultats et les limites de ce modèle avant de conclure.

Ce projet regroupe ce calepin et des [annexes](#Annexes), qui visent à fournir une documentation approfondie sur la description des forces impliquées dans le modèle, ou à documenter les détails techniques de l'implémentation des fonctions supports ne faisant pas à proprement parler du modèle mais permettent d'enrichir les fonctionnalités du projet sans allourdir le calepin principal.

## Modèle et méthode

Pour expliquer la trajectoire de l'ISS au cours du temps, nous nous référons au **principe fondamental de la dynamique (PFD)** qui nous indique que dans un référentiel galiléen, l'accélération du centre d'inertie $\vec a$ d'un système de masse $m$ constante est proportionnelle à la résultante des forces $\vec F_i$ qu'il subit, et inversement proportionnelle à $m$, soit :
$$\vec a = \sum\frac{\vec F_i}m$$

L'addition des compostantes de l'accélération issues de chaque force exterieure nous donne l'expression de l'équation différentielle du mouvement du centre d'inertie. 
Toutefois, le nombre de forces impliquées étant théoriquement quasi-infini  (tout corps dans l'univers exercant une force de gravitation non nulle) mais également d'ordres de grandeur fortement inégaux, il convient d'effectuer un choix rigoureux des forces à modéliser. Notre modèle vise donc à prendre en compte les forces s'exerçant sur la station par ordre d'influence sur son accélération, en commençant par les plus influentes.

Pour sélectionner les forces à modéliser, nous nous appuyons sur la figure suivante, issue de l'ouvrage *Satellite Orbits*, de Montenbruck & Gill [[2]](#ref-2), qui détaille les ordres de grandeurs des différentes forces agissant sur un satellite en orbite terrestre.

![](resources/Montenbruck.jpg)

Ainsi, en tenant compte de l'ordre de grandeur de chaque force excercée sur un satellite en orbite basse autour de 400 km d'altitude (6 771 km du centre de la Terre), nous sélectionnons les forces suivantes :

- la force gravitationnelle de la Terre,
- la force géopotentielle $J_{2,0}$ et $J_{2,2}$,
- la force de traînée atmosphérique,
- la force gravitationnelle de la Lune,
- la force gravitationnelle du Soleil.

Dans la suite de cette section, nous modélisons chacune de ces forces, afin d'en déduire leurs contributions à l'accélération de l'ISS, que nous modélisons comme un point matériel situé à l'emplacement de son centre de masse. L'équation différentielle du mouvement est ensuite résolue dans la section suivante de façon numérique pour définir une trajectoire de résultat.

À titre informatif, les seules forces représentant une contribution supérieure à $10^{-10}\ \mathrm{km.s^{-2}}$ pour l'accélération et qui n'ont pas été prises en compte sont : 
- le coefficient $J_{6,6}$ du géopotentiel, qui représente un apport nettement moins significatif que $J_{2,0}$ et $J_{2,2}$ ;
- les forces de marées, c'est-à-dire la déformation de la croûte terrestre dûe aux forces de gravitation extérieures, puisque les variations de densité de la croûte terrestre sont en partie modélisées dans l'expression du géopotentiel.


Pour implémenter numériquement de façon efficace chaque force de notre modèle, nous les faisons hériter d'une classe abstraite `Force`, dont les spécificités sont détaillées en [Annexe : Modèle d'implémentation des forces](%5BAnnexe%5D%20Mod%C3%A8le%20d'impl%C3%A9mentation%20des%20forces.ipynb).

### Force de gravitation terrestre

La force gravitationnelle exercée par la Terre, représentée $GM$ sur la figure des forces, est la force principale s'exerçant sur un satellite en orbite basse. Son expression ci-dessous considère la Terre comme une sphère parfaite, ce qui revient à un point materiel situé au centre d'une sphère parfaite.

In [1]:
from isslib import Force

class EarthGravity(Force):
    """Force de gravitation terrestre."""

    @property
    def name(self):
        """Nom de la force sous forme de string."""
        return "Force de gravitation terrestre"
    
    @property
    def formula(self):
        """Expression de la force au format LaTeX."""
        return r"-\frac{GMm}{r^3}\mathbf r"

    def acceleration(self, u, t):
        """EDO gravitation terrestre."""
        # vecteur position        
        r = np.array(u[:3])
        # Expression de l'acceleration résultante de la force
        return - self.G * self.earth_mass / np.linalg.norm(r)**3 * r

EarthGravity()

Force de gravitation terrestre

Avec $G$ la constante universelle de gravitation, $M$ la masse de la Terre, $r$ la distance entre les deux centres de masse et $\mathbf r$ le vecteur position du satellite, sa contribution à l'accélération de ce dernier est :
$$\ddot r_{gT} = -\frac{GM}{r^3} \mathbf{r}$$

### Force géopotentielle

Si l'on considère que la masse de la Terre n'est pas uniformément répartie dans une sphère parfaite, on peut choisir d'intégrer le potentiel $U$ de la gravité terrestre pour modéliser sa force d'attraction sur un satellite. On a alors :
$$\ddot r_{gp}=\nabla U$$
avec $U = \dfrac{GM}r$.
Si on généralise l'expression à une masse arbitraire : 
$$ U = \int \frac{\rho(s)d^3s}{|r - s|}$$
avec ${\rho(s)}$ la densité à un point $s$ au sein de la Terre, et $|r - s|$ la distance du satellite au point $s$.

En développant avec la méthode des polynômes de Legendre, on obtient :
$$ \ddot r_{gp} = \nabla\frac{GM}r \cdot \sum_{n=0}^\inf \sum_{m=0}^n \frac{R^n}{r^n} \bar{P}_{nm}\sin\phi\left(\bar{C}_{nm} \cos(m\lambda) + \bar{S}_{nm} \sin(m\lambda)\right)
$$
où chaque combinaison de $n,m$ est représenté sous la forme $J_{n,m}$.

On constate alors que $J_{0,0}$ donne l'expression de la force de gravitation terrestre. **La modélisation indépendante de la gravitation terrestre est alors inutile, son influence étant représenté par le géopotentiel.**

[TODO] Annulation des 1

On modélise les coefficients $J_{2,0}$ et $J_{2,2}$ qui sont, en plus de la gravitation terrestre, les principales influences gravitationnelles en orbite basse. On a alors :

$$ \ddot r_{gp} = \nabla\frac{GM}{r} J_{0 ,0} + \nabla\frac{GM}{r} J_{2 ,0} + \nabla\frac{GM}{r} J_{2 ,2} $$

In [2]:
from astropy.time import Time
import astropy.units as units
from isslib.coordinates import celestial2terrestrial
import pandas as pd

class Geopotential(Force):
    def __init__(self, t0=pd.Timestamp.now()):
        self.t0 = Time(t0)

    @property
    def C20(self):
        """Valeur du coefficient de Legendre cosinus de degré 2 d'ordre 0."""
        return -484.2e-6  # coefficient sans unité

    @property
    def C22(self):
        """Valeur du coefficient de Legendre cosinus de degré 2 d'ordre 2."""
        return 2.439261e-6  # coefficient sans unité

    @property
    def S22(self):
        """Valeur du coefficient de Legendre sinus de degré 2 d'ordre 2."""
        return -1.400266e-6  # coefficient sans unité

    @property
    def name(self):
        """Nom de la force sous forme de string."""
        return "Force géopotentielle"
    
    @property
    def formula(self):
        """Expression de la force au format LaTeX."""
        return (r"\nabla\frac{GMm}r\left[1+\frac{\sqrt5}2\frac{R^2}{r^2}"
                r"\Bigg(\bar{C}_{2,0}\left(3\sin^2(\varphi)-1\right)+"
                r"\sqrt3\cos^2(\varphi)\left(\bar{C}_{2,2}\cos(2\lambda)+"
                r"\bar{S}_{2,2}\sin(2\lambda)\right)\Bigg)\right]")

    def acceleration(self, u, t):
        """EDO géopotentiel."""
        # vecteur position        
        r = np.array(u[:3])
        # calcul de la distance au centre de la terre (norme de r),
        # longitude et latitude terrestre à la date et l'heure donnée
        datetime = self.t0 + t * units.s
        n_r, longitude, latitude = celestial2terrestrial(*r, datetime,
                                                         mode='spherical')
        # nabla(GM/r)
        nablaGMr = - self.G * self.earth_mass / n_r**3 * r
        # Expression de l'acceleration résultante de la force
        return nablaGMr * (1 + np.sqrt(5)*self.earth_radius**2/(2*n_r**2)
                           * (self.C20 * (3 * np.sin(latitude)**2 - 1)
                              + np.sqrt(3) * np.cos(latitude)**2
                              * (self.C22 * np.cos(2*longitude)
                                 + self.S22 * np.sin(2*longitude))))
Geopotential()

Force géopotentielle

Sa contribution à l'accélération du satellite est alors :

$$
\boxed{
\ddot{r} = -\frac{GM}{r^3}\left[1+\frac{\sqrt5}2\frac{R^2}{r^2}\Bigg(\bar{C}_{2,0}\left(3\sin^2(\varphi)-1\right)+\sqrt3\cos^2(\varphi)\left(\bar{C}_{2,2}\cos(2\lambda)+\bar{S}_{2,2}\sin(2\lambda)\right)\Bigg)\right] 
\mathbf r}
$$

où $r$ est la distance entre les deux centres de gravité, $R$ le rayon de la Terre, **$\lambda$** la longitude et $\phi$ la latitude du satellite dans le repère sphérique.

La force géopotentielle est développé plus en détail dans l'[annexe : Force Géopotentielle](%5BAnnexe%5D%20Force%20g%C3%A9opotentielle%20WIP.ipynb).

In [3]:
#### Damien en est là.

### Force de traînée atmosphérique

La deuxième force d'importance capitale lorsque l'on cherche à modéliser la trajectoire d'un satellite en basse altitude est la force de traînée atmosphérique. Elle représente la perturbation non gravitationnelle la plus importante sur les satellites à basse altitude. Il y a pourtant plusieurs difficultés à sa modélisation : les propriétés physiques de l'atmosphère, et notamment sa densité, ne sont pas connues de façon très précises. Nous allons ici procéder à plusieurs simplifications. Premièrement, nous allons supposer que la hauteur du satellite est constante par rapport au centre de la Terre, et qu'à cette hauteur, il n'y a pas de variation conséquente de la physique atmosphérique, ce qui permet de négliger les variations de densité et de température. Nous allons enfin supposer que la surface que présente l'ISS au flux de particules qu'elle rencontre est constante. 

Ainsi, la force de traînée à plusieurs propriétés : 
- elle est dirigée de façon opposée à la vitesse, selon le flux atmosphérique, et contribue donc à ralentir le satellite
- elle dépend de la surface du satellite qui rencontre le flux de particule, et  du coefficient de traînée qui lui est attaché
- elle dépend de sa masse
- elle dépend de la densité atmosphérique

Si l'on considère un élément de masse $\Delta m$ d'une colonne d'air qui percute l'aire transversale $A$ du satellite durant l'intervalle $dt$, on peut écrire : 
$$\Delta m = \rho A v_r \Delta t$$ 

Si $v_r$ est la vitesse relative du satellite par rapport à l'atmopshère, et ou $\rho$ est la densité atmosphérique. L'impulsion $dp$ exercée sur le satellite est alors : 
$$\Delta p = \Delta m v_r = \rho A {v_r}^2 \textbf{e}_v$$

Que l'on peut mettre en relation avec la force résultante avec la formule 
$$ F = \frac{\Delta p } {\Delta t}$$

Au final, on obtient

$$ \boxed{\ddot{r} = -\frac{1}{2} C_D \frac{A}{m} \rho {v_r}^2 \textbf{e}_v}$$

Avec $m$ la masse du satellite, $C_D$ le coefficient de traînée, sans dimension, qui décrit les interactions de l'atmopshère avec la surface du satellite, et qui varie globalement entre 1.5 et 3. La direction de l'accélération est toujours opposée à celle de la vitesse, et $$ \mathbf{e}_v = \mathbf{v}_r / v_r$$

La connaissance de $C_D$ n'est pas parfaite, cependant, des études et des simulations numériques ont permis d'estimer une valeur moyenne du coefficient de traînée pour l'ISS. En général, on considère que le coefficient de traînée pour l'ISS est compris entre 1,2 et 2,2, avec une valeur typique de 1,5.

La vitesse relative du satellite par rapport à l'atmosphère peut être approximée en supposant que l'atmosphère tourne à la même vitesse que la Terre.

On a alors $$\textbf{v}_r = \textbf{v} - \mathbf{\omega r}$$

Avec $\textbf{v}$ la vélocité du satellite, $\textbf{r}$ le vecteur position, et $\omega$ la vitesse angulaire de la Terre de taille $0,7292.10^{-4} rad.s^{-1}$

Enfin, $\rho$ est la masse volumique atmosphérique à hauteur de l'ISS. Elle est de l'ordre de $10^{-12}$ environ à cette altitude (NASA). [modèle NRLMSISE-00 (Navy-Remote-Lower-Mesosphere-Space-Model-Earth-2000), développé par la US Navy et utilisé pour estimer les propriétés de l'atmosphère à différentes altitudes.]

Avec ces données et l'expression de l'accélération, nous pouvons ajouter cette force à notre équation différentielle.

La méthode générale est expliquée plus en détail dans le **Notebook *Force - Traînée atmosphérique***.

### Force de gravitation lunaire

L'une des forces à ne pas négliger est celle de la Lune, qui a une masse bien supérieur à celle du satellite, et qui influe donc sur sa trajectoire, bien qu'à un ordre moindre que les deux forces précédentes.

Pour obtenir l'accélération d'un satellite par rapport à la Lune dans un référentiel géocentrique, il faut considérer l'accélération de la Terre par rapport à la Lune, et pas seulement l'accélération du satellite par rapport à elle.

On a donc 
$$
\ddot{r} = GM_L\left( \frac{L-r}{|L-r|^3} - \frac{L}{|L|^3} \right)
$$

Avec $L$ la coordonnée de la Lune de masse $M_L$ par rapport au système de coordonnées géocentriques et $r$ celle du satellite.

Comme la lune est bien plus loin que les satellites (et surtout l'ISS) de la Terre, on peut se servir d'approximations : 

$$
\frac{1}{|L-r|^3} = {\frac{1}{s^2+r^2-2Lr(\mathbf{e_Le_r})}^{3/2} \approx \frac{1}{L^3} \left(1+3\frac{r}{L}(\mathbf{e_Le_r})\right)}
$$

Ici, $\mathbf{e_L} = \dfrac{\mathbf{L}}{L}$ et $\mathbf{e_r} = \dfrac{\mathbf{r}}{r}$ sont les vecteurs unitaires, pointant respectivement vers $L$ la Lune et $r$.

On a alors la relation approximative suivante : 
$$
\boxed{\mathbf{\ddot{r}} \approx \frac{GM_Lr}{L^3}\big(-\mathbf{e_r}+3\mathbf{e_L}(\mathbf{e_L}\mathbf{e_r})\big)}
$$


### Force de gravitation solaire

Enfin, la dernière force que nous avons souhaité prendre en compte est celle de la gravitation solaire. En influant sur la trajectoire de la Terre, elle modifie aussi celle de l'ISS.

En suivant la même méthode que pour la force de gravitation solaire, on obtient
$$ 
\boxed{\mathbf{\ddot{r}} \approx \frac{GM_sr}{s^3}\big(-\mathbf{e}_r+3\mathbf{e}_s(\mathbf{e}_s\mathbf{e}_r)\big)}
$$

Avec $s$ la coordonnée du Soleil par rapport au système de coordonnées géocentriques.

La méthode générale développée pour ces deux dernières forces est expliquée plus en détail dans le **Notebook *Force - Gravitation***


### Modélisation

Les forces que nous avons choisi de modéliser, afin de visualiser l'effet qu'elles ont sur l'ISS, sont donc à présent mises en équation. Ces 4 équations différentielles vont nous permettre d'obtenir la trajectoire de l'ISS, en intégrant à l'aide de méthodes numériques et à partir de coordonnées initiales. Nous développerons sur ces derniers points dans la section **Intégration numérique**.

---
---
Nous pourrons alors écrire l'équation différentielle de notre problème. En intégrant, nous obtiendrons alors un modèle de la vitesse, puis de la trajectoire de l'ISS, en tenant compte des paramètres initiaux. Exprimée en coordonnées cylindriques, basées sur la longitude et la latitude, nous comparerons alors notre modèle aux données collectées par la NASA, afin de conclure sur la pertinence de notre modèle, et sur l'intérêt de modéliser uniquement forces que nous avons sélectionnées. Nous serons alors en mesure de présenter les difficultés que nous avons rencontrées, les limites de nos travaux, ainsi que l'intérêt de ce projet en termes d'apprentissage.


Dans cette partie, nous allons expliciter les formules de ces dernières, ainsi que les suppositions que nous avons faites pour les appliquer à notre modèle. 

---
---

Exprimée en coordonnées cylindriques, basées sur la longitude et la latitude, nous comparerons alors notre modèle aux données collectées par la NASA, afin de conclure sur la pertinence de notre modèle, et sur l'intérêt de modéliser uniquement forces que nous avons sélectionnées. Nous serons alors en mesure de présenter les difficultés que nous avons rencontrées, les limites de nos travaux, ainsi que l'intérêt de ce projet en termes d'apprentissage.

---
---

## Données sources
[TODO] Explication des données NASA

## Intégration numérique
[TODO] Explication de l'intégration des EDO

## Visualisation des résultats
[TODO] Rédaction

## Discussion
[TODO] Observations et limites du modèle

En comparant les données sources au modèle, on constate donc que les forces choisies ne suffisent pas à reproduire l'orbite de l'ISS. En effet, le modèle ne parvient pas à reproduire les variations latitudinales de l'orbite de l'ISS. De même, on constate que les variations d'altitude sont plus importantes dans le modèle que pour les données réelles.

Plusieurs facteurs peuvent expliquer ces changements : 

Premièrement, les simplifications faites au niveau de la traînée atmosphérique ne permettent pas une prise en compte exhaustive de cette force, qui change pourtant considérablement la trajectoire d'un satellite. En effet, ne pas prendre en compte les variations de températures et de géographie, et supposer que la densité de l'air est homogène, est une simplification conséquente. 

De même, lors du calcul du géopotentiel, nous avons choisi de nous arrêter aux coefficients les plus bas, mais il aurait pu être intéressant de chercher à en calculer davantage. 

Enfin, les nombreuses forces que nous n'avons pas modélisé, comme la pression de radiation du soleil ou de l'atmosphère, l'albédo, la marée terrestre, ou encore les forces de gravitation produites par les autres corps célestes, ont aussi une influence sur la trajectoire d'un satellite, bien que l'ordre de grandeur qui leur est associé est moins important.



## Conclusion
Apprentissage et ouvertures de cette modélisation

## Annexes

## Références et bibliographie


* <a class="anchor" name="ref-1">[1]</a> International Space Station Facts and Figures ; NASA ; https://www.nasa.gov/feature/facts-and-figures ; 4 janvier 2023.
*<a class="anchor" name="ref-2">[2]</a> Satellite Orbits: Models, Methods, and Applications ; Montenbruck Oliver et Gill Eberhard ; Springer Berlin Heidelberg ; 2000.


* R. Roy, Orbital Motion (CRC Press, 2005), p. 117-118.
* D.A. Vallado et al., Fundamentals of Astrodynamics and Applications, 4th ed. (Microcosm Press, 2013), p. 48-49.
* "Aerodynamic Characteristics of the International Space Station (ISS) for Orbital Altitudes" (2003) par J. J. Bertin, T. K. Stanton, et R. G. Johnson. Cette étude a utilisé des simulations numériques pour étudier les caractéristiques aérodynamiques de l'ISS à différentes altitudes orbitales, et a estimé un coefficient de traînée moyen de 1,5 pour l'ISS.

* "Aerodynamic Analysis of the International Space Station" (2003) par S. K. Agrawal et S. S. Joshi. Cette étude a utilisé une combinaison de simulations numériques et de données expérimentales pour étudier la traînée atmosphérique subie par l'ISS, et a estimé un coefficient de traînée moyen de 1,8 pour l'ISS.
